# Regression Case Study
-  Data : Diamonds from sns
-  How variables affect DV
-  Extracted from Linked In post
-  statistical tests and regression model on diamonds dataset from seaborn library with important concepts to data, including: Data cleaning, Data Exploration, Data visualization, OLS Regression model, Hypothesis testing including: ANOVA, Post hoc test, ANCOVA, MANOVA testing.

In [ ]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
#data 
df = sns.load_dataset('diamonds')
df.info()

In [ ]:
df.head()

In [ ]:
# take log of price
df['log_price'] = np.log(df['price'])
df.head(2)

In [ ]:
#subset columns
df1 = df[['color','price','log_price']]
df1.tail(2)

In [ ]:
df1.info()

In [ ]:
df1.columns

In [ ]:
df1.shape

In [ ]:
df1.describe()
#high sd for price

In [ ]:
df1.price.plot(kind='hist')

In [ ]:
df1.price.plot(kind='kde')

In [ ]:
sns.displot(data=df1, x = 'price', kde = True)

In [ ]:
#missing values - Nil
df1.isnull().sum()

In [ ]:
df1.head(3)

In [ ]:
colorCount = df1.color.value_counts().reset_index().rename(columns={'color':'count','index':'color'})
colorCount

In [ ]:
# Categorical Values 
fig = px.bar(colorCount, x='color',y='count' )
fig.show();

In [ ]:
#df1.loc[df['pop'] < 2.e6, 'country'] = 'Other countries' # Represent only large countries
fig = px.pie(colorCount, names='color', values='count', title='Colors and their Count : Pie Chart')
fig.show();

In [ ]:
# Mean price per color
meanPriceColor = df1.groupby(['color'], as_index=False)['price'].agg(np.mean)
print(meanPriceColor)
fig = px.bar(meanPriceColor, x='color',y='price', color='color', title='Mean Price per Color' )
fig.show();

In [ ]:
# box plot
fig = px.box(df1, x="color", y="price")
fig.show()

In [ ]:
sns.boxplot(data=df1, x='color', y='price')

In [ ]:
df1['price'].describe()

In [ ]:
q1=df1[['price']].quantile(0.25)
q3=df1[['price']].quantile(0.75)
IQR=q3-q1
IQR, q1, q3

In [ ]:
q1 - IQR, q3 + IQR

In [ ]:
#outliers.sum()

In [ ]:
df1.query('price < -3425')

In [ ]:
df1.query('price > 9868')
#5355

In [ ]:
df1.loc[df1['price'] > 9868, :]

In [ ]:
df2 = df1.loc[df1['price'] <= 9868, :]
df2.shape

In [ ]:
np.where(df1[(df1['price'] <= 9868)])

In [ ]:
# Regression 
import statsmodels.api as st
from statsmodels.formula.api import ols

In [ ]:
olsmodel = ols(formula ='log_price ~ C(color)', data=df1).fit()
# we wanted to treat explicitly as categorical, we could have done so by using the C() operator:

In [ ]:
olsmodel.summary()

In [ ]:
#anova
st.stats.anova_lm(olsmodel, type=2)

-  t test to state : mean is equal among 2 groups of data.
-  does color affect the price of diamong
-  p value tells if can reject Ho : p < .05 : Significant difference due to color -> Reject Ho : Color affect the mean price
-  F value tells how much color is affect the outcome (log_price)

In [ ]:
df.columns

In [ ]:
#add another column, ie cut 
df1 = df1.join(df['cut'])
df1.head(3)

In [ ]:
sns.boxplot(data=df1, x='cut', y='log_price')

In [ ]:
sns.barplot(data=df1.cut.value_counts().reset_index().rename(columns={'cut':'count','index':'cut'}), x='cut',y='count')

In [ ]:
df1.groupby(['cut'], as_index=False)['log_price'].mean()

In [ ]:
sns.barplot(data=df1.groupby(['cut'], as_index=False)['log_price'].mean(), x='cut',y='log_price')
#almost same with cut type

In [ ]:
#second model
olsmodel2 = ols(formula = 'log_price ~ C(color) + C(cut) + C(color):C(cut)', data=df1).fit()

In [ ]:
olsmodel2.summary()

## hypothesis setting
- Color
    -  Ho : No difference in price due to color, Ha : Difference in price due to color
- Cut
    -  Ho : No difference in price due to cut, Ha : Difference in price due to cut
-  Color + Cut Interaction/ Combination
    -  Ho : No difference in price due to cut & color, Ha : Difference in price due to cut & color



In [ ]:
st.stats.anova_lm(olsmodel2, type=2)

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
tukeyhsd = pairwise_tukeyhsd(endog = df1['log_price'], groups = df1['color'], alpha = .05)

In [ ]:
tukeyhsd.summary()
# Reject Column
##  True - Reject Ho in favour of Ha
##  False - Accept Ho

## Ho
-  Ho : No Difference in price due to colors only
    -  D & E
    -  F & G
-  Rest have difference in prices due to colors

In [ ]:
# add carat to the model
df1 = df1.join(df['carat'])
df1.head()

In [ ]:
olsmodel3 = ols('log_price ~ C(color) + carat', df1).fit()
olsmodel3.summary()

In [ ]:
#ancova
ancova_table = st.stats.anova_lm(olsmodel3, type=2)
ancova_table

In [ ]:
sns.lmplot(data = df1, x='carat', y='log_price', hue='color').set(title = 'ANCOVA plot with multiple covariates')

In [ ]:
#!pip install pingouin
from pingouin import ancova

In [ ]:
ancova(data=df1, dv='log_price', covar='carat', between='color')

### Ancova summary
-   p-unc (uncorrected) < .05,-> Reject Ho that each color results in same avg log_price even after controlling carat variable
-   Conclude that color itself affect the price of diamong

In [ ]:
df1.groupby('color')['log_price'].describe()

In [ ]:
df1.groupby('color')['carat'].describe()

In [ ]:
sns.boxplot(data=df1, x='color', y='carat')
#outliers seen

In [ ]:
df1.carat.hist()

In [ ]:
# MANOVA
from statsmodels.multivariate.manova import MANOVA
fit = MANOVA.from_formula('log_price + carat ~ color', data =df1)
print(fit.mv_test())

In [ ]:
# p value < .005 
# strong association between color & caret + log_price 

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda

In [ ]:
sampleDF1 = df1.sample(n=100, random_state=32190).reset_index()
sampleDF1.shape

In [ ]:
sampleDF1.head(2)

In [ ]:
X = sampleDF1[['log_price', 'carat']]
y = sampleDF1['color']
X.shape, y.shape

In [ ]:
sampleDF1.isnull().sum()

In [ ]:
X_ne = pd.DataFrame(lda().fit(X=X, y=y).transform(X), columns=['lda1', 'lda2'])

In [ ]:
X_ne.head()

In [ ]:
X_ne['color'] = sampleDF1[['color']]
X_ne.head(2)

In [ ]:
sns.scatterplot(data= X_ne, x='lda1', y='lda2', hue='color')

# End here
-  https://www.linkedin.com/feed/update/urn:li:activity:7088692799499362304/